# Polygonization

In this workshop we will learn how to convert our topological model of the building into a geometric model.

In [2]:
import topogenesis as tg
import numpy as np 
import os

In [3]:
two = 2 ** np.arange(8)
print(two)

[  1   2   4   8  16  32  64 128]


In [23]:
s = tg.create_stencil("von_neumann", 1, 1)
s.set_index((0,0,0), 0)

stencil([[[0, 0, 0],
          [0, 1, 0],
          [0, 0, 0]],

         [[0, 1, 0],
          [1, 0, 1],
          [0, 1, 0]],

         [[0, 0, 0],
          [0, 1, 0],
          [0, 0, 0]]])

In [24]:
l_bis = []
for i in range(2**8):
    bi = np.array(list(np.binary_repr(i, width=8))).astype(int).reshape((2,2,2))
    l_bi = tg.to_lattice(bi, [0,0,0])
    l_bis.append(l_bi)

In [ ]:
for l in l_bis:
    for i in range(8):
        pass

### Credits

In [ ]:
__aut__author__ = "Shervin Azadi"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Polygonization"